In [1]:
import pandas as pd
import numpy as np
import os
from pandas import DataFrame
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import h5py
import gc
import seaborn as sns
import warnings
from scipy import signal
from sklearn.preprocessing import label_binarize
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split,GridSearchCV,LeaveOneOut,cross_val_score,StratifiedKFold
from sklearn import svm,datasets
from sklearn.svm import SVC
from itertools import cycle
from sklearn.metrics import f1_score,classification_report,confusion_matrix, precision_score,recall_score,roc_auc_score, roc_curve, auc, precision_recall_curve
import scipy
from scipy.stats import pearsonr,spearmanr
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import optuna
import catboost as cb
from catboost import CatBoostClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split



mean_fpr = np.linspace(0, 1, 100)
def readfilez(file):  

    f = h5py.File(file,'r')
    f.keys() 
    eegz=f['z'][:]   
#     print(np.shape(eegz))
    eeg1=np.array(eegz[:,:30]).astype(np.float16)
    eeg2=np.array(eegz[:,30:60]).astype(np.float16)
    emg=np.array(eegz[:,60:90]).astype(np.float16)
    scores=np.array(eegz[:,90:91]).astype(np.float16)
    f.close()
    gc.collect()
    return emg,eeg1,eeg2,scores
#求解敏感性、特异性
plt.rcParams['font.sans-serif'] = ['SimHei']



    
def sen(Y_test,Y_pred,n):#n为分类数
    
    sen = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        sen1 = tp / (tp + fn)
        sen.append(sen1)
        
    return sen
def spe(Y_test,Y_pred,n):
    
    spe = []
    con_mat = confusion_matrix(Y_test,Y_pred)
    for i in range(n):
        number = np.sum(con_mat[:,:])
        tp = con_mat[i][i]
        fn = np.sum(con_mat[i,:]) - tp
        fp = np.sum(con_mat[:,i]) - tp
        tn = number - tp - fn - fp
        spe1 = tn / (tn + fp)
        spe.append(spe1)
    
    return spe

def fill_plot(interp_tpr,name,aucs,path):
    colors = ['aqua', 'green', 'red']   
    target_names = ['Wake', 'NREM', 'REM']
    interp_tpr=np.array(interp_tpr)    
    fig, ax = plt.subplots(figsize=(8, 8))
    mean_tpr = np.mean(interp_tpr[:,0,:], axis=0)
    mean_tpr[-1] = 1.0
    std_tpr = np.std(interp_tpr[:,0,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[0],
        label=f"Mean ROC of '{target_names[0]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[0], alpha=0.2,label=r"95% Confidence interval")  
#     ax.fill_between(
#         mean_fpr,
#         tprs_lower,
#         tprs_upper,
#         color=colors[0],
#         alpha=0.2,
#         label=r"$\pm$ 1 std. dev.",
#     )
    
    mean_tpr = np.mean(interp_tpr[:,1,:], axis=0)
    std_tpr = np.std(interp_tpr[:,1,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[1],
        label=f"Mean ROC of '{target_names[1]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
#     ax.fill_between(
#         mean_fpr,
#         tprs_lower,
#         tprs_upper,
#         color=colors[1],
#         alpha=0.2,
#         label=r"$\pm$ 1 std. dev.",
#     )    
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[1], alpha=0.2,label=r"95% Confidence interval")  
    
    
    mean_tpr = np.mean(interp_tpr[:,2,:], axis=0)
    std_tpr = np.std(interp_tpr[:,2,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[2],
        label=f"Mean ROC of '{target_names[2]}'(Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
#     ax.fill_between(
#         mean_fpr,
#         tprs_lower,
#         tprs_upper,
#         color=colors[2],
#         alpha=0.2,
#         label=r"$\pm$ 1 std. dev.",
#     )       
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[2], alpha=0.1,label=r"95% Confidence interval")   
    
    ax.set(
        xlim=[-0.05, 1.00],
        ylim=[-0.05, 1.2],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Mean ROC curve with variability\n(Positive label '{target_names[0],target_names[1],target_names[2]}')",
    )
#     ax.axis("square")

    ax.legend(loc="lower right")
    plt.savefig(path+'/{}'.format(name),dpi=600,bbox_inches = 'tight')
    plt.show()
    


In [11]:

#lgb 调参
def tiaocanlgb(freq,sh,canshu,nl,lr,ns,md,ss,cb,ra,rl):
    x_true=np.array(freq)
    y_true=sh.astype(np.int)
    x_trainy, y_trainy=x_true, y_true
    le = LabelEncoder()
    y_trainy = le.fit_transform(y_trainy)
    print(y_trainy)
    print(x_trainy.shape)
    cv_params =canshu
    other_params = {'num_leaves':3,'learning_rate': 0.11, 'n_estimators': 100, 'max_depth': 4, 
                     'subsample': 0.7, 'colsample_bytree': 0.1, 'reg_alpha': 0.03, 'reg_lambda': 0.04}
     #'tree_method':'gpu_hist'
    model = lgb.LGBMClassifier(**other_params)
    model.fit(x_trainy,y_trainy)            #1-3进行数据训练与预测
    print(model.score(x_trainy,y_trainy))
    #model = xgb.XGBClassifier()
    kflod = StratifiedKFold(n_splits=5, shuffle = True,random_state=7)#将训练/测试数据集划分10个互斥子集，,n_jobs = 10
    optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, cv=kflod,scoring = 'neg_log_loss',verbose=10)
    optimized_GBM.fit(x_trainy, y_trainy)
    evalute_result = optimized_GBM.cv_results_['mean_test_score']
    print('每轮迭代运行结果:{0}'.format(evalute_result))
    print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
    print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
    return optimized_GBM.best_params_

def canshulgb(eeg2f,tab):
    freq=np.array(eeg2f)
    print(np.shape(freq))
    sh=np.array(tab)
    print(np.shape(sh))
    nl,lr,ns,md,ss,cb,ra,rl=3,0.11,100,4,0.7,0.1,0.03,0.04
    ra=list(tiaocanlgb(freq,sh,{'reg_alpha':np.arange(0.1,1,0.1)},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    rl=list(tiaocanlgb(freq,sh,{'reg_lambda':np.arange(0.1,1,0.1)},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    md=list(tiaocanlgb(freq,sh,{'max_depth':range(3,8,1)},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    lr=list(tiaocanlgb(freq,sh,{'learning_rate':np.arange(0.01,0.3,0.01)},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    nl=list(tiaocanlgb(freq,sh,{'num_leaves':[5,6,7,12,13,14,15,28,29,30,31]},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    ns=list(tiaocanlgb(freq,sh,{'n_estimators':range(100,1000,100)},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    cb=list(tiaocanlgb(freq,sh,{'colsample_bytree':np.arange(0.3,1,0.1)},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    ss=list(tiaocanlgb(freq,sh,{'subsample':np.arange(0.7,1.2,0.1)},nl,lr,ns,md,ss,cb,ra,rl).values())[0]
    return nl,lr,ns,md,ss,cb,ra,rl

#调整参数画图
def tcplotlgb(eeg2f,eeg2fyc,tab,tabyc,lgbcan,name):
    nl,lr,ns,md,ss,cb,ra,rl=lgbcan
    #大脑
    ez=eeg2f
    ezyc=eeg2fyc
    le = LabelEncoder()
    tab = le.fit_transform(tab)
    tabyc = le.fit_transform(tabyc)
    x_train,y_train=np.array(ez),np.array(tab)
    x_test,y_test=np.array(ezyc),np.array(tabyc)
    model = lgb.LGBMClassifier(num_leaves=nl,learning_rate= lr, n_estimators=ns, max_depth= md,
                            subsample= ss, colsample_bytree= cb,reg_alpha=ra,reg_lambda=rl)

    model.fit(x_train, y_train)
    # x_test, y_test = ros.fit_resample(x_test, y_test)
    pred=model.predict(x_test)
    preds=model.predict_proba(x_test)
    score = model.score(x_test, y_test)
    print(y_test)
    print(pred)
    # print(preds)
    target_names = ['Wake', 'NREM', 'REM']

    y_test1 = label_binarize(y_test, classes=[0, 1, 2])

    fpr={}
    tpr={}
    roc_auc={}
    rocz=[]
    f_head2=[]
    interp_tprf=[]
    eegx=list(i for i in np.arange(0,30))
    eegx = [str(x) for x in eegx]
    EEG_head=[name]
    for i in EEG_head:
        f_head2.extend(list(i+'Cerebellar EEG'+j for j in eegx))
        f_head2.extend(list(i+'Cerebral EEG'+j for j in eegx))
        f_head2.extend(list(i+'EMG'+j for j in eegx))
    
    pres=[]
    recal=[]
    sens=[]
    spec=[]
    importa=[]
    pres.append(precision_score(y_test, pred,average=None))
    recal.append(recall_score(y_test, pred,average=None))
    sens.append(sen(y_test, pred,3))
    spec.append(spe(y_test, pred,3))
    impo=pd.Series(model.feature_importances_, index=f_head2)
    importa.append(impo)

    for i in range(3):
    #         print(i)
        fpr[i], tpr[i], _ = roc_curve(y_test1[:, i], preds[:, i])
        rocz.append(auc(fpr[i], tpr[i]))
        roc_auc[i] = auc(fpr[i], tpr[i])
        print("roc_auc:",roc_auc)
        interp_tprff=np.interp(mean_fpr, fpr[i], tpr[i])
        interp_tprf.append(interp_tprff)
    # Plot all ROC curves
    plt.figure()
    lw = 2
    colors = cycle(['darkorange','cornflowerblue','DeepPink'])
    for i, color in zip(range(3), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of {0} (area = {1:0.2f})'
                 ''.format(target_names[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
#     plt.show()
    pres=np.array(pres)
    recal=np.array(recal)
    sens=np.array(sens)
    spec=np.array(spec)
    df=pd.DataFrame({'精确率Wakefulness':pres[:,0],'精确率Nrem':pres[:,1],'精确率Rem':pres[:,2],
                     '召回率Wakefulness':recal[:,0],'召回率Nrem':recal[:,1],'召回率Rem':recal[:,2],
                     '敏感性Wakefulness':sens[:,0],'敏感性Nrem':sens[:,1],'敏感性Rem':sens[:,2],
                     '特异性Wakefulness':spec[:,0],'特异性Nrem':spec[:,1],'特异性Rem':spec[:,2]})
#     df.to_excel(f_sv+"{}report.xls".format(name))
    df1=pd.DataFrame(importa)
#     df1.to_csv(f_sv+"{}importa.csv".format(name))
    return interp_tprf,score,df,df1
def xunlianlgb(f_h5z,f_sv,num,nl,lr,ns,md,ss,cb,ra,rl):
    interp_tprd=[]
    interp_tprx=[]
    interp_tprj=[]
    dfd,dfd1=[],[]
    dfx,dfx1=[],[]
    dfj,dfj1=[],[]
    aucd,aucx,aucj=[],[],[]
    scored,scorex,scorej=[],[],[]
    dfd=pd.DataFrame()
    dfd1=pd.DataFrame()
    dfx=pd.DataFrame()
    dfx1=pd.DataFrame()
    dfj=pd.DataFrame()
    dfj1=pd.DataFrame()
    for j in range(50):
        info=os.listdir(f_h5z)
        print(info)
        infoxl = np.random.choice(info,num, replace=False)
        infoyc =np.setdiff1d(info, infoxl)
        tab=[]
        eeg1f,eeg2f,emgf=[],[],[]
        for i in range(np.shape(infoxl)[0]):
#             print(infoxl[i])
            print("xxxxxxxxxx")
            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,infoxl[i])
#             print(infor)
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1f.extend(eeg1)
            eeg2f.extend(eeg2)
            emgf.extend(emg)
            tab.extend(scores)
        ez= np.concatenate((eeg1f, eeg2f, emgf), axis=1)
        #预测数据
        print(infoyc)
        tabyc=[]
        eeg1fyc,eeg2fyc,emgfyc=[],[],[]
        
        for i in range(np.shape(infoyc)[0]):
#             print(info[i])
            print("xxxxxxxxxx")
            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,info[i])
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1fyc.extend(eeg1)
            eeg2fyc.extend(eeg2)
            emgfyc.extend(emg)
            tabyc.extend(scores)
        ezyc= np.concatenate((eeg1fyc, eeg2fyc, emgfyc), axis=1)
        print("here")
        print(np.shape(eeg1f))
        #大脑
        #调整参数
        interp_tprl,score,df,df1=tcplotlgb(ez,ezyc,tab,tabyc,nl,lr,ns,md,ss,cb,ra,rl,"Lightgbm")
        interp_tprd.append(interp_tprl)
        scored.append(score)
        
#         df=pd.DataFrame(df)
#         df1=pd.DataFrame(df1)
        dfd = pd.concat([dfd, df])
        dfd1 = pd.concat([dfd1, df1])
        print("wo",dfd)
    print("ni",dfd)
    fill_plot(interp_tprd,"{}Lightgbm".format(num),scored,f_sv)
    dfd.to_csv(f_sv+"{}Lightgbm_report.csv".format(num))
    dfd1.to_csv(f_sv+"{}Lightgbm_importa.csv".format(num))


In [12]:
#xgboost 调参
def tiaocanxgb(freq,sh,canshu,ra,rl,md,lr,ga,mw,ns,mp,ce,se):
    x_true=np.array(freq)
    y_true=sh.astype(np.int)
    x_trainy, y_trainy=x_true, y_true
    le = LabelEncoder()
    y_trainy = le.fit_transform(y_trainy)
    print(y_trainy)
    print(x_trainy.shape)
    cv_params =canshu
    other_params = {'objective':'multi:softmax','num_class':3,'learning_rate': 0.11, 'n_estimators': 100, 'max_depth': 4, 'min_child_weight': 4, 'seed': 7,
                     'subsample': 0.7, 'colsample_bytree': 0.1, 'gamma': 0.4, 'reg_alpha': 0.03, 'reg_lambda': 0.04,'max_delta_step':1,
                    'tree_method':'gpu_hist'}
     #'tree_method':'gpu_hist'
    model = xgb.XGBClassifier(**other_params)
    model.fit(x_trainy,y_trainy)            #1-3进行数据训练与预测
    print(model.score(x_trainy,y_trainy))
    #model = xgb.XGBClassifier()
    kflod = StratifiedKFold(n_splits=5, shuffle = True,random_state=7)#将训练/测试数据集划分10个互斥子集，,n_jobs = 10
    optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, cv=kflod,scoring = 'neg_log_loss',verbose=10)
    optimized_GBM.fit(x_trainy, y_trainy)
    evalute_result = optimized_GBM.cv_results_['mean_test_score']
    print('每轮迭代运行结果:{0}'.format(evalute_result))
    print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
    print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
    return optimized_GBM.best_params_

def canshuxgb(eeg2f,tab):
    freq=np.array(eeg2f)
    print(np.shape(freq))
    sh=np.array(tab)
    print(np.shape(sh))
    ra,rl,md,lr,ga,mw,ns,mp,ce,se=0.1,0.1,3,0.1,0.1,1,100,1,0.1,0.1
    ra=list(tiaocanxgb(freq,sh,{'reg_alpha':np.arange(0.1,1,0.1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    rl=list(tiaocanxgb(freq,sh,{'reg_lambda':np.arange(0.1,1,0.1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    md=list(tiaocanxgb(freq,sh,{'max_depth':range(3,8,1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    lr=list(tiaocanxgb(freq,sh,{'learning_rate':np.arange(0.01,0.3,0.01)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    ga=list(tiaocanxgb(freq,sh,{'gamma':np.arange(0.1,0.9,0.1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    mw=list(tiaocanxgb(freq,sh,{'min_child_weight':range(1,5,1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    ns=list(tiaocanxgb(freq,sh,{'n_estimators':range(100,250,10)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    mp=list(tiaocanxgb(freq,sh,{'max_delta_step':range(1,5,1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    ce=list(tiaocanxgb(freq,sh,{'colsample_bytree':np.arange(0.3,1,0.1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    se=list(tiaocanxgb(freq,sh,{'subsample':np.arange(0.7,1.2,0.1)},ra,rl,md,lr,ga,mw,ns,mp,ce,se).values())[0]
    return ra,rl,md,lr,ga,mw,ns,mp,ce,se

#调整参数画图
def tcplotxgb(eeg2f,eeg2fyc,tab,tabyc,xgbcan,name):
    ra,rl,md,lr,ga,mw,ns,mp,ce,se=xgbcan
    #大脑
    ez=eeg2f
    ezyc=eeg2fyc
    le = LabelEncoder()
    tab = le.fit_transform(tab)
    tabyc = le.fit_transform(tabyc)
    x_train,y_train=np.array(ez),np.array(tab)
    x_test,y_test=np.array(ezyc),np.array(tabyc)


    # model = svm.SVC(probability=True,C=0.4,gamma=0.2,  kernel='rbf')#默认参数
    model = xgb.XGBClassifier(objective='multi:softmax',num_class=3,learning_rate= lr, n_estimators=ns, max_depth= md, min_child_weight= mw,
                            subsample= se, colsample_bytree= ce, gamma= ga,max_delta_step=md,reg_alpha=ra,reg_lambda=rl,tree_method='gpu_hist')

    model.fit(x_train, y_train)
    # x_test, y_test = ros.fit_resample(x_test, y_test)
    pred=model.predict(x_test)
    preds=model.predict_proba(x_test)
    score = model.score(x_test, y_test)
    print(y_test)
    print(pred)
    # print(preds)
    target_names = ['Wake', 'NREM', 'REM']

    y_test1 = label_binarize(y_test, classes=[0, 1, 2])

    fpr={}
    tpr={}
    roc_auc={}
    rocz=[]
    f_head2=[]
    interp_tprf=[]
    eegx=list(i for i in np.arange(0,30))
    eegx = [str(x) for x in eegx]
    EEG_head=[name]
    for i in EEG_head:
        f_head2.extend(list(i+'Cerebellar EEG'+j for j in eegx))
        f_head2.extend(list(i+'Cerebral EEG'+j for j in eegx))
        f_head2.extend(list(i+'EMG'+j for j in eegx))
    pres=[]
    recal=[]
    sens=[]
    spec=[]
    importa=[]
    pres.append(precision_score(y_test, pred,average=None))
    recal.append(recall_score(y_test, pred,average=None))
    sens.append(sen(y_test, pred,3))
    spec.append(spe(y_test, pred,3))
    impo=pd.Series(model.feature_importances_, index=f_head2)
    importa.append(impo)

    for i in range(3):
    #         print(i)
        fpr[i], tpr[i], _ = roc_curve(y_test1[:, i], preds[:, i])
        rocz.append(auc(fpr[i], tpr[i]))
        roc_auc[i] = auc(fpr[i], tpr[i])
        print("roc_auc:",roc_auc)
        interp_tprff=np.interp(mean_fpr, fpr[i], tpr[i])
        interp_tprf.append(interp_tprff)
    # Plot all ROC curves
    plt.figure()
    lw = 2
    colors = cycle(['darkorange','cornflowerblue','DeepPink'])
    for i, color in zip(range(3), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of {0} (area = {1:0.2f})'
                 ''.format(target_names[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
#     plt.show()
    pres=np.array(pres)
    recal=np.array(recal)
    sens=np.array(sens)
    spec=np.array(spec)
    df=pd.DataFrame({'精确率Wakefulness':pres[:,0],'精确率Nrem':pres[:,1],'精确率Rem':pres[:,2],
                     '召回率Wakefulness':recal[:,0],'召回率Nrem':recal[:,1],'召回率Rem':recal[:,2],
                     '敏感性Wakefulness':sens[:,0],'敏感性Nrem':sens[:,1],'敏感性Rem':sens[:,2],
                     '特异性Wakefulness':spec[:,0],'特异性Nrem':spec[:,1],'特异性Rem':spec[:,2]})
#     df.to_excel(f_sv+"{}report.xls".format(name))
    df1=pd.DataFrame(importa)
#     df1.to_csv(f_sv+"{}importa.csv".format(name))
    return interp_tprf,score,df,df1

def xunlianxgb(f_h5z,f_sv,num,xgbcan):
    interp_tprd=[]
    interp_tprx=[]
    interp_tprj=[]
    dfd,dfd1=[],[]
    dfx,dfx1=[],[]
    dfj,dfj1=[],[]
    aucd,aucx,aucj=[],[],[]
    scored,scorex,scorej=[],[],[]
    dfd=pd.DataFrame()
    dfd1=pd.DataFrame()
    dfx=pd.DataFrame()
    dfx1=pd.DataFrame()
    dfj=pd.DataFrame()
    dfj1=pd.DataFrame()
    for j in range(50):
        info=os.listdir(f_h5z)
        print(info)
        infoxl = np.random.choice(info,num, replace=False)
        infoyc =np.setdiff1d(info, infoxl)
        tab=[]
        eeg1f,eeg2f,emgf=[],[],[]
        for i in range(np.shape(infoxl)[0]):
#             print(infoxl[i])
            print("xxxxxxxxxx")
            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,infoxl[i])
#             print(infor)
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1f.extend(eeg1)
            eeg2f.extend(eeg2)
            emgf.extend(emg)
            tab.extend(scores)
        ez= np.concatenate((eeg1f, eeg2f, emgf), axis=1) 
        #预测数据
        print(infoyc)
        tabyc=[]
        eeg1fyc,eeg2fyc,emgfyc=[],[],[]
        
        for i in range(np.shape(infoyc)[0]):
#             print(info[i])
            print("xxxxxxxxxx")
            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,info[i])
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1fyc.extend(eeg1)
            eeg2fyc.extend(eeg2)
            emgfyc.extend(emg)
            tabyc.extend(scores)
        ezyc= np.concatenate((eeg1fyc, eeg2fyc, emgfyc), axis=1)    
        print("here")
        print(np.shape(eeg1f))
        #大脑
        #调整参数
        interp_tprl,score,df,df1=tcplotxgb(ez,ezyc,tab,tabyc,xgbcan,"XGBoost")
        interp_tprd.append(interp_tprl)
        scored.append(score)
        
#         df=pd.DataFrame(df)
#         df1=pd.DataFrame(df1)
        dfd = pd.concat([dfd, df])
        dfd1 = pd.concat([dfd1, df1])
        print("wo",dfd)

    print("ni",dfd)
    fill_plot(interp_tprd,"{}XGBoost".format(num),scored,f_sv)
    dfd.to_csv(f_sv+"{}XGBoost_report.csv".format(num))
    dfd1.to_csv(f_sv+"{}XGBoost_importa.csv".format(num))

In [5]:
#catboost 调参
def tiaocancat(freq,sh,canshu,it,lr,dh):
    x_true=np.array(freq)
    y_true=sh.astype(int)
    x_trainy, y_trainy=x_true, y_true
    le = LabelEncoder()
    y_trainy = le.fit_transform(y_trainy)
    print(y_trainy)
    print(x_trainy.shape)
    cv_params =canshu
    other_params = {'iterations': 1000,'learning_rate': 0.01, 'depth': 4, 'task_type':'GPU'}
    model = CatBoostClassifier(**other_params)
    model.fit(x_trainy,y_trainy)            #1-3进行数据训练与预测
    print(model.score(x_trainy,y_trainy))
    #model = xgb.XGBClassifier()
    kflod = StratifiedKFold(n_splits=5, shuffle = True,random_state=7)#将训练/测试数据集划分10个互斥子集，,n_jobs = 10
    optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, cv=kflod,scoring = 'neg_log_loss',verbose=10)
    optimized_GBM.fit(x_trainy, y_trainy)
    evalute_result = optimized_GBM.cv_results_['mean_test_score']
    print('每轮迭代运行结果:{0}'.format(evalute_result))
    print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
    print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))
    return optimized_GBM.best_params_

def canshucat(eeg2f,tab):
    freq=np.array(eeg2f)
    print(np.shape(freq))
    sh=np.array(tab)
    print(np.shape(sh))
    it,lr,dh=1000,0.01,4
    it=list(tiaocancat(freq,sh,{'iterations':np.arange(500,1500,50)},it,lr,dh).values())[0]
    lr=list(tiaocancat(freq,sh,{'learning_rate':np.arange(0.01,1,0.05)},it,lr,dh).values())[0]
    dh=list(tiaocancat(freq,sh,{'depth':range(5,15,1)},it,lr,dh).values())[0]

    return it,lr,dh

#调整参数画图
def tcplotcat(eeg2f,eeg2fyc,tab,tabyc,catcan,name):
    it,lr,dh=catcan
    #大脑
    ez=eeg2f
    ezyc=eeg2fyc
    le = LabelEncoder()
    tab = le.fit_transform(tab)
    tabyc = le.fit_transform(tabyc)
    x_train,y_train=np.array(ez),np.array(tab)
    x_test,y_test=np.array(ezyc),np.array(tabyc)

    # model = svm.SVC(probability=True,C=0.4,gamma=0.2,  kernel='rbf')#默认参数
    model = CatBoostClassifier(iterations=it,learning_rate=lr,depth=dh,task_type='GPU')

    model.fit(x_train, y_train)
    # x_test, y_test = ros.fit_resample(x_test, y_test)
    pred=model.predict(x_test)
    preds=model.predict_proba(x_test)
    score = model.score(x_test, y_test)
    print(y_test)
    print(pred)
    # print(preds)
    target_names = ['Wake', 'NREM', 'REM']

    y_test1 = label_binarize(y_test, classes=[0, 1, 2])

    fpr={}
    tpr={}
    roc_auc={}
    rocz=[]
    f_head2=[]
    interp_tprf=[]
    eegx=list(i for i in np.arange(0,30))
    eegx = [str(x) for x in eegx]
    EEG_head=[name]
    for i in EEG_head:
        f_head2.extend(list(i+'Cerebellar EEG'+j for j in eegx))
        f_head2.extend(list(i+'Cerebral EEG'+j for j in eegx))
        f_head2.extend(list(i+'EMG'+j for j in eegx))
    pres=[]
    recal=[]
    sens=[]
    spec=[]
    importa=[]
    pres.append(precision_score(y_test, pred,average=None))
    recal.append(recall_score(y_test, pred,average=None))
    sens.append(sen(y_test, pred,3))
    spec.append(spe(y_test, pred,3))
    impo=pd.Series(model.feature_importances_, index=f_head2)
    importa.append(impo)

    for i in range(3):
    #         print(i)
        fpr[i], tpr[i], _ = roc_curve(y_test1[:, i], preds[:, i])
        rocz.append(auc(fpr[i], tpr[i]))
        roc_auc[i] = auc(fpr[i], tpr[i])
        print("roc_auc:",roc_auc)
        interp_tprff=np.interp(mean_fpr, fpr[i], tpr[i])
        interp_tprf.append(interp_tprff)
    # Plot all ROC curves
    plt.figure()
    lw = 2
    colors = cycle(['darkorange','cornflowerblue','DeepPink'])
    for i, color in zip(range(3), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of {0} (area = {1:0.2f})'
                 ''.format(target_names[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
#     plt.show()
    pres=np.array(pres)
    recal=np.array(recal)
    sens=np.array(sens)
    spec=np.array(spec)
    df=pd.DataFrame({'精确率Wakefulness':pres[:,0],'精确率Nrem':pres[:,1],'精确率Rem':pres[:,2],
                     '召回率Wakefulness':recal[:,0],'召回率Nrem':recal[:,1],'召回率Rem':recal[:,2],
                     '敏感性Wakefulness':sens[:,0],'敏感性Nrem':sens[:,1],'敏感性Rem':sens[:,2],
                     '特异性Wakefulness':spec[:,0],'特异性Nrem':spec[:,1],'特异性Rem':spec[:,2]})
#     df.to_excel(f_sv+"{}report.xls".format(name))
    df1=pd.DataFrame(importa)
#     df1.to_csv(f_sv+"{}importa.csv".format(name))
    return interp_tprf,score,df,df1

In [21]:
def xl_all(f_h5z,f_sv,num,lgbcan,xgbcan,catcan):
    xgb_ra,xgb_rl,xgb_md,xgb_lr,xgb_ga,xgb_mw,xgb_ns,xgb_mp,xgb_ce,xgb_se=xgbcan
    lgb_nl,lgb_lr,lgb_ns,lgb_md,lgb_ss,lgb_cb,lgb_ra,lgb_rl=lgbcan
    cat_it,cat_lr,cat_dh=catcan
    interp_tprd_lgb=[]
    dfd_xgb,dfd1_lgb=[],[]
    scored_lgb=[]
    dfd_lgb=pd.DataFrame()
    dfd1_lgb=pd.DataFrame()
    
    interp_tprd_xgb=[]
    dfd_xgb,dfd1_xgb=[],[]
    scored_xgb=[]
    dfd_xgb=pd.DataFrame()
    dfd1_xgb=pd.DataFrame()

    interp_tprd_cat=[]
    dfd_cat,dfd1_cat=[],[]
    scored_cat=[]
    dfd_cat=pd.DataFrame()
    dfd1_cat=pd.DataFrame()
    for j in range(20):
        info=os.listdir(f_h5z)
        print(info)
        infoxl = np.random.choice(info,num, replace=False)
        infoyc =np.setdiff1d(info, infoxl)
        tab=[]
        eeg1f,eeg2f,emgf=[],[],[]
        for i in range(np.shape(infoxl)[0]):
#             print(infoxl[i])
            print("xxxxxxxxxx")
            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,infoxl[i])
#             print(infor)
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1f.extend(eeg1)
            eeg2f.extend(eeg2)
            emgf.extend(emg)
            tab.extend(scores)
        ez= np.concatenate((eeg1f, eeg2f, emgf), axis=1) 
        #预测数据
        print(infoyc)
        tabyc=[]
        eeg1fyc,eeg2fyc,emgfyc=[],[],[]
        
        for i in range(np.shape(infoyc)[0]):
#             print(info[i])
            print("xxxxxxxxxx")
            domain = os.path.abspath(f_h5z)
            infor = os.path.join(domain,info[i])
            emg,eeg1,eeg2,scores=readfilez(infor)
            eeg1fyc.extend(eeg1)
            eeg2fyc.extend(eeg2)
            emgfyc.extend(emg)
            tabyc.extend(scores)
        ezyc= np.concatenate((eeg1fyc, eeg2fyc, emgfyc), axis=1)    
        print("here")
        print(np.shape(eeg1f))
        #大脑
        #调整参数
        interp_tprl,score,df,df1=tcplotxgb(ez,ezyc,tab,tabyc,xgbcan,"XGBoost")        
        interp_tprd_xgb.append(interp_tprl)
        scored_xgb.append(score)
        dfd_xgb= pd.concat([dfd_xgb, df])
        dfd1_xgb= pd.concat([dfd1_xgb, df1])
        print("wo",dfd_xgb)
        interp_tprl,score,df,df1=tcplotlgb(ez,ezyc,tab,tabyc,lgbcan,"LGBoost")
        interp_tprd_lgb.append(interp_tprl)
        scored_lgb.append(score)
        dfd_lgb= pd.concat([dfd_lgb, df])
        dfd1_lgb= pd.concat([dfd1_lgb, df1])
        print("wo",dfd_lgb)
        
        interp_tprl,score,df,df1=tcplotcat(ez,ezyc,tab,tabyc,catcan,"CATBoost")
        interp_tprd_cat.append(interp_tprl)
        scored_cat.append(score)
        dfd_cat= pd.concat([dfd_cat, df])
        dfd1_cat= pd.concat([dfd1_cat, df1])
        print("wo",dfd_lgb)        
#         df=pd.DataFrame(df)
#         df1=pd.DataFrame(df1)
    dfd_lgb.to_csv(f_sv+"{}LGBoost_report.csv".format(num))
    dfd1_lgb.to_csv(f_sv+"{}LGBoost_importa.csv".format(num))
    dfd_xgb.to_csv(f_sv+"{}XGBoost_report.csv".format(num))
    dfd1_xgb.to_csv(f_sv+"{}XGBoost_importa.csv".format(num))
    dfd_cat.to_csv(f_sv+"{}CATBoost_report.csv".format(num))
    dfd1_cat.to_csv(f_sv+"{}CATBoost_importa.csv".format(num))    
    return interp_tprd_lgb,scored_lgb,interp_tprd_xgb,scored_xgb,interp_tprd_cat,scored_cat
#     fill_plot_all(interp_tprd_lgb,"{}LGBoost".format(num),scored_lgb,interp_tprd_xgb,"{}XGBoost".format(num),scored_xgb,f_sv)


In [18]:
def fill_plot_all(interp_tpr_lgb,aucs_lgb,interp_tpr_xgb,aucs_xgb,path):
#LGBoost
    colors = ['aqua', 'green', 'red']   
    target_names = ['Wake', 'NREM', 'REM']
    interp_tpr=np.array(interp_tpr_lgb)
    aucs=aucs_lgb
    fig, ax = plt.subplots(figsize=(8, 8))
    mean_tpr = np.mean(interp_tpr[:,0,:], axis=0)
    mean_tpr[-1] = 1.0
    std_tpr = np.std(interp_tpr[:,0,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[0],
        label=f"Mean ROC of '{target_names[0]}' in LGboost (Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[0], alpha=0.2,label=r"95% Confidence interval")  
    
    mean_tpr = np.mean(interp_tpr[:,1,:], axis=0)
    std_tpr = np.std(interp_tpr[:,1,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[1],
        label=f"Mean ROC of '{target_names[1]}' in LGboost (Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
 
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[1], alpha=0.2,label=r"95% Confidence interval")  
    
    
    mean_tpr = np.mean(interp_tpr[:,2,:], axis=0)
    std_tpr = np.std(interp_tpr[:,2,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        color=colors[2],
        label=f"Mean ROC of '{target_names[2]}' in LGboost (Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[2], alpha=0.1,label=r"95% Confidence interval")   
    
# xgboost
    colors = ['aqua', 'green', 'red']   
    target_names = ['Wake', 'NREM', 'REM']
    interp_tpr=np.array(interp_tpr_xgb)    
    aucs=aucs_xgb
#     fig, ax = plt.subplots(figsize=(8, 8))
    mean_tpr = np.mean(interp_tpr[:,0,:], axis=0)
    mean_tpr[-1] = 1.0
    std_tpr = np.std(interp_tpr[:,0,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
#     ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
    ax.plot(
        mean_fpr,
        mean_tpr,
        dashes=[6, 2],
        color=colors[0],
        label=f"Mean ROC of '{target_names[0]}' in XGboost (Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[0], alpha=0.1,label=r"95% Confidence interval")  
    
    mean_tpr = np.mean(interp_tpr[:,1,:], axis=0)
    std_tpr = np.std(interp_tpr[:,1,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        dashes=[6, 2],
        color=colors[1],
        label=f"Mean ROC of '{target_names[1]}' in XGboost (Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
 
    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[1], alpha=0.1,label=r"95% Confidence interval")  
    
    
    mean_tpr = np.mean(interp_tpr[:,2,:], axis=0)
    std_tpr = np.std(interp_tpr[:,2,:], axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)    
    ax.plot(
        mean_fpr,
        mean_tpr,
        dashes=[6, 2],
        color=colors[2],
        label=f"Mean ROC of '{target_names[2]}' in XGboost (Mean AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    conf_intveral = scipy.stats.norm.interval(0.95,mean_tpr,std_tpr)
    ax.fill_between(mean_fpr, conf_intveral[0], conf_intveral[1], color=colors[2], alpha=0.1,label=r"95% Confidence interval")   
    
    ax.set(
        xlim=[-0.05, 1.00],
        ylim=[-0.05, 1.2],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Mean ROC curve with variability\n(Positive label '{target_names[0],target_names[1],target_names[2]}')",
    )
#     ax.axis("square")
    ax.legend(loc="lower right")
    plt.savefig(path+'/model_roc_plot',dpi=600,bbox_inches = 'tight')
    plt.show()